In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd



### Plan

- write function to get soup from each page (i.e. 10 results)
    - for each article, extract: title, link, year, site, first author
    - NB if returns empty page, stop the for loop
- do this for 3 links: MRz, MRs, BC
    - exclude all articles that are not in BC (all papers citing THE paper but are not actually on BC)

In [29]:
def scrape_url(url):
    # get header: http://myhttpheader.com/
    headers = {'User-Agent':
           'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.115 Safari/537.36'}

    response = requests.get(url,headers=headers)
    soup = BeautifulSoup(response.content,'lxml')
    
  
    df_list = parse_soup(soup)
    return df_list
    
def parse_soup(soup):
    df_list = []
    
    num_articles = len(soup.select('[data-lid]'))
    
    for item in soup.select('[data-lid]'):
        try:
            # extarct from HTML
            title = [item.select('h3')[0].get_text()]
            title = [x.replace('[HTML][HTML] ', "") for x in title]
            link = [item.select('a')[0]['href']]
            abstract = [item.select('.gs_rs')[0].get_text()]
            info = item.select('.gs_a')[0].get_text()
            
            # parse info
            info_split = info.split("-")
            journal_year = info_split[1].split(", ")
            first_author = [info_split[0].split(", ")[0]]
        
            info_list = [*info_split, *journal_year , *first_author]
            info_list = [x.strip(' ') for x in info_list]
            info_list = [x.replace('\xa0', "") for x in info_list]
            article = [*title, info_list[4], info_list[5], info_list[2], info_list[3],  *link]
            
        except Exception as e:
            #raise e 
            print('==cant parse this soup')
            article=[]
            
        df_list.append(article)  
          
    return(df_list)
            
            

In [30]:
url='https://scholar.google.com/scholar?start=20&q=mendelian+randomization&hl=en&as_sdt=2005&sciodt=0,5&cites=6613969377995102680&scipsc=1'
df_list = scrape_url(url)
df_list
df = pd.DataFrame(df_list, columns=['title', 'year', 'first_author', 'source_site', 'source_name', 'link'])
df

,title,year,first_author,source_site,source_name,link
0,"Allergy, asthma, and the risk of breast and pr...",2020,X Jiang,Springer,Cancer Causes &…,https://link.springer.com/article/10.1007/s105...
1,Reading Mendelian randomisation studies: a gui...,2018,NM Davies,bmj.com,Bmj,https://www.bmj.com/content/bmj/362/bmj.k601.f...
2,Can the impact of childhood adiposity on disea...,2019,TG Richardson,medrxiv.org,medRxiv,https://www.medrxiv.org/content/medrxiv/early/...
3,Polycystic ovary syndrome is causally associat...,2020,PF Wu,ajog.org,American Journal of…,https://www.ajog.org/article/S0002-9378(20)305...
4,Mendelian randomization analyses of geneticall...,2020,S Li,nature.com,NPJ precision…,https://www.nature.com/articles/s41698-020-001...
5,A comprehensive re-assessment of the associati...,Suen,X Han,"Nature…, 2021",nature.com,https://www.nature.com/articles/s41467-020-203...
6,Health-related effects of education level: a M...,2020,S Yuan,medrxiv.org,MedRxiv,https://www.medrxiv.org/content/10.1101/2020.0...
7,Genetically raised circulating bilirubin level...,Torres,N Murphy…,"Cells, 2021",mdpi.com,https://www.mdpi.com/2073-4409/10/2/394/pdf
8,Assessing a causal relationship between circul...,2019,KE Johnson,biorxiv.org,bioRxiv,https://www.biorxiv.org/content/biorxiv/early/...
9,Cancer risks in patients with vitiligo: a Mend...,2020,Y Wen,Springer,Journal of cancer…,https://link.springer.com/article/10.1007/s004...


In [59]:
df_list